In [49]:
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autotime
import mne
import numpy as np
import pandas as pd
from tqdm import tqdm
from numba import njit, jit, float32
import matplotlib.pyplot as plt
plt.style.use('classic')
from BrainPulse import (dataset,
                        vector_space,
                        distance_matrix,
                        recurrence_quantification_analysis,
                        features_space,
                        plot)

def normalize(tSignal):
    # copy the data if needed, omit and rename function argument if desired
    signal = np.copy(tSignal) # signal is in range [a;b]
    signal -= np.min(signal) # signal is in range to [0;b-a]
    signal /= np.max(signal) # signal is normalized to [0;1]
    signal -= 0.5 # signal is in range [-0.5;0.5]
    signal *=2 # signal is in range [-1;1]
    return signal

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 2.32 ms (started: 2022-07-18 22:25:06 +02:00)


In [38]:
cut_freq = 60
win_len = 240
n_fft = 512
percentile = 35
s_rate = 512

time: 450 µs (started: 2022-07-18 14:17:02 +02:00)


In [3]:
evoked, epochs = dataset.eegMCI_data_epochs(stim_type1='AV',stim_type2='A', subject=2, run_type='test.dat_2', path_to_data_folder='auditory_visual_audiovisual')

Not setting metadata
50 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
Not setting metadata
50 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
['Cz', 'CPz', 'POz', 'Pz', 'P1', 'P2', 'C3', 'C4', 'O1', 'O2', 'T7', 'T8', 'P3', 'P4', 'F3', 'F4']
Not setting metadata
2 matching events found
No baseline correction applied
0 projection items activated
0 bad epochs dropped
time: 116 ms (started: 2022-07-18 13:53:04 +02:00)


In [5]:
epochs

[<EpochsArray |  50 events (all good), -0.199219 - 0.998047 sec, baseline off, ~3.8 MB, data loaded,
  '1': 50>,
 <EpochsArray |  50 events (all good), -0.199219 - 0.998047 sec, baseline off, ~3.8 MB, data loaded,
  '1': 50>]

time: 2.1 ms (started: 2022-07-18 13:53:25 +02:00)


In [ ]:
evoked

# compute epochs raw

In [6]:
epochs[0].get_data()[0].shape

(16, 614)

time: 1.15 ms (started: 2022-07-18 13:54:29 +02:00)


In [39]:
matrix_list_tdemb_epoch = []
matrix_list_stft_epoch = []

def compute_epoch_rqa(epochs):

    for electrode_index in range(16):

        electrode_signal = epochs.get_data()[0][electrode_index]* 1e6
        stft_close = vector_space.compute_stft((electrode_signal),
                                               n_fft=n_fft, win_len=win_len,
                                               s_rate=s_rate,
                                               cut_freq=cut_freq)

        matrix_stft = distance_matrix.EuclideanPyRQA_RP_stft(stft_close)
        nbr_stft=np.percentile(matrix_stft, percentile)
        matrix_binary_stft = distance_matrix.set_epsilon(matrix_stft,nbr_stft)

        matrix_tdemb = distance_matrix.EuclideanPyRQA_RP(electrode_signal, embedding=5, timedelay=9)
        nbr_tdemb = np.percentile(matrix_tdemb, percentile)
        matrix_binary_tdemb = distance_matrix.set_epsilon(matrix_tdemb, nbr_tdemb)

        matrix_list_tdemb_epoch.append(matrix_binary_tdemb)
        matrix_list_stft_epoch.append(matrix_binary_stft)

time: 1.14 ms (started: 2022-07-18 14:17:09 +02:00)


In [40]:
compute_epoch_rqa(epochs[0])

time: 440 ms (started: 2022-07-18 14:17:13 +02:00)


In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=4,figsize=(8,8),dpi=200)
ax_ravel = axs.ravel()

for i,ax in enumerate(ax_ravel):
    # matrix_to_plot = matrix_list_stft_epoch[i]
    matrix_to_plot = matrix_list_tdemb_epoch[i]

    ax.imshow(matrix_to_plot, cmap='Greys', origin='lower')
    ax.set_xticks(np.linspace(0, matrix_to_plot.shape[0] , ax.get_xticks().shape[0]))
    ax.set_yticks(np.linspace(0, matrix_to_plot.shape[0] , ax.get_xticks().shape[0]))
    ax.set_xticklabels([str(np.around(x,decimals=1)) for x in np.linspace(-0.2, (matrix_to_plot.shape[0] / s_rate)-0.2, ax.get_xticks().shape[0])])
    ax.set_yticklabels([str(np.around(x, decimals=1)) for x in np.linspace(-0.2, (matrix_to_plot.shape[0] / s_rate)-0.2, ax.get_yticks().shape[0])])


    ax.set_title(str(epochs[0].ch_names[i]))
    ax.set_xlabel('time (s)')
    ax.set_ylabel('time (s)')
    ax.xaxis.grid()
    ax.yaxis.grid()
plt.suptitle('TDEMB epoch td = 9, emb = 5')
fig.tight_layout()


In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=4,figsize=(8,8),dpi=200)
ax_ravel = axs.ravel()

for i,ax in enumerate(ax_ravel):
    matrix_to_plot = matrix_list_stft_epoch[i]
    # matrix_to_plot = matrix_list_tdemb_epoch[i]

    ax.imshow(matrix_to_plot, cmap='Greys', origin='lower')
    ax.set_xticks(np.linspace(0, matrix_to_plot.shape[0] , ax.get_xticks().shape[0]))
    ax.set_yticks(np.linspace(0, matrix_to_plot.shape[0] , ax.get_xticks().shape[0]))
    ax.set_xticklabels([str(np.around(x,decimals=1)) for x in np.linspace(-0.2, (matrix_to_plot.shape[0] / s_rate)-0.2, ax.get_xticks().shape[0])])
    ax.set_yticklabels([str(np.around(x, decimals=1)) for x in np.linspace(-0.2, (matrix_to_plot.shape[0] / s_rate)-0.2, ax.get_yticks().shape[0])])


    ax.set_title(str(epochs[0].ch_names[i]))
    ax.set_xlabel('time (s)')
    ax.set_ylabel('time (s)')
    ax.xaxis.grid()
    ax.yaxis.grid()
plt.suptitle('STFT epoch nfft = 512, win len ='+str(win_len))
fig.tight_layout()

## compute for ERP


In [16]:
matrix_list_tdemb_evoked = []
matrix_list_stft_evoked = []

def compute_evoked_rqa(evoked):

    for electrode_index in range(16):

        electrode_signal = evoked[electrode_index]* 1e6
        stft_close = vector_space.compute_stft((electrode_signal),
                                               n_fft=n_fft, win_len=win_len,
                                               s_rate=s_rate,
                                               cut_freq=cut_freq)

        matrix_stft = distance_matrix.EuclideanPyRQA_RP_stft(stft_close)
        nbr_stft=np.percentile(matrix_stft, percentile)
        matrix_binary_stft = distance_matrix.set_epsilon(matrix_stft,nbr_stft)

        matrix_tdemb = distance_matrix.EuclideanPyRQA_RP(electrode_signal, embedding=3, timedelay=9)
        nbr_tdemb = np.percentile(matrix_tdemb, percentile)
        matrix_binary_tdemb = distance_matrix.set_epsilon(matrix_tdemb, nbr_tdemb)

        matrix_list_tdemb_evoked.append(matrix_binary_tdemb)
        matrix_list_stft_evoked.append(matrix_binary_stft)

time: 676 µs (started: 2022-07-18 13:59:41 +02:00)


In [17]:
compute_evoked_rqa(evoked.get_data()[0])

time: 429 ms (started: 2022-07-18 13:59:47 +02:00)


In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=4,figsize=(8,8),dpi=200)
ax_ravel = axs.ravel()

for i,ax in enumerate(ax_ravel):
    # matrix_to_plot = matrix_list_stft_evoked[i]
    matrix_to_plot = matrix_list_tdemb_evoked[i]

    ax.imshow(matrix_to_plot, cmap='Greys', origin='lower')
    ax.set_xticks(np.linspace(0, matrix_to_plot.shape[0] , ax.get_xticks().shape[0]))
    ax.set_yticks(np.linspace(0, matrix_to_plot.shape[0] , ax.get_xticks().shape[0]))
    ax.set_xticklabels([str(np.around(x,decimals=1)) for x in np.linspace(-0.2, (matrix_to_plot.shape[0] / s_rate)-0.2, ax.get_xticks().shape[0])])
    ax.set_yticklabels([str(np.around(x, decimals=1)) for x in np.linspace(-0.2, (matrix_to_plot.shape[0] / s_rate)-0.2, ax.get_yticks().shape[0])])


    ax.set_title(str(epochs[0].ch_names[i]))
    ax.set_xlabel('time (s)')
    ax.set_ylabel('time (s)')
    ax.xaxis.grid()
    ax.yaxis.grid()
    # ax.xaxis.grid(True, which='minor')
plt.suptitle('TDEMB evoked td = 9, emb = 5')
fig.tight_layout()


In [ ]:
fig, axs = plt.subplots(nrows=4, ncols=4,figsize=(8,8),dpi=200)
ax_ravel = axs.ravel()

for i,ax in enumerate(ax_ravel):
    matrix_to_plot = matrix_list_stft_evoked[i]
    # matrix_to_plot = matrix_list_tdemb_evoked[i]

    ax.imshow(matrix_to_plot, cmap='Greys', origin='lower')
    ax.set_xticks(np.linspace(0, matrix_to_plot.shape[0] , ax.get_xticks().shape[0]))
    ax.set_yticks(np.linspace(0, matrix_to_plot.shape[0] , ax.get_xticks().shape[0]))
    ax.set_xticklabels([str(np.around(x,decimals=1)) for x in np.linspace(-0.2, (matrix_to_plot.shape[0] / s_rate)-0.2, ax.get_xticks().shape[0])])
    ax.set_yticklabels([str(np.around(x, decimals=1)) for x in np.linspace(-0.2, (matrix_to_plot.shape[0] / s_rate)-0.2, ax.get_yticks().shape[0])])


    ax.set_title(str(epochs[0].ch_names[i]))
    ax.set_xlabel('time (s)')
    ax.set_ylabel('time (s)')
    ax.xaxis.grid()
    ax.yaxis.grid()
plt.suptitle('STFT evoked nfft = 512, win len ='+str(win_len))
fig.tight_layout()